## Import necessary packages and modules

In [17]:
import os
import cv2
import torch
import numpy as np
import imutils
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision.models import resnet18
from torch import nn
from torch.utils.data import Dataset
from torchvision import transforms
from torch.utils.data import DataLoader
from torch.optim import lr_scheduler
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report, confusion_matrix

In [ ]:
net = resnet18()

In [ ]:
net.conv1 = nn.Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)


In [18]:
class BrainTumorDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.image_list = []
        self.label_list = []
        self.class_names = [d for d in os.listdir(root_dir) if os.path.isdir(os.path.join(root_dir, d))]

        for label, class_name in enumerate(self.class_names):
            class_folder = os.path.join(root_dir, class_name)
            image_files = [f for f in os.listdir(class_folder) if os.path.isfile(os.path.join(class_folder, f))]
            self.image_list.extend([os.path.join(class_folder, f) for f in image_files])
            self.label_list.extend([label]*len(image_files))

    def __len__(self):
        return len(self.image_list)

    def __getitem__(self, idx):
        img_name = self.image_list[idx]
        img = cv2.imread(img_name)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img = crop_img(img)
        
        img = transforms.ToTensor()(img)
        
        if self.transform:
            img = self.transform(img)

        label = self.label_list[idx]
        
        return img, label

In [19]:
def crop_img(img):
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    gray = cv2.GaussianBlur(gray, (3, 3), 0)

    thresh = cv2.threshold(gray, 45, 255, cv2.THRESH_BINARY)[1]
    thresh = cv2.erode(thresh, None, iterations=2)
    thresh = cv2.dilate(thresh, None, iterations=2)

    cnts = cv2.findContours(thresh.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    cnts = imutils.grab_contours(cnts)
    c = max(cnts, key=cv2.contourArea)

    extLeft = tuple(c[c[:, :, 0].argmin()][0])
    extRight = tuple(c[c[:, :, 0].argmax()][0])
    extTop = tuple(c[c[:, :, 1].argmin()][0])
    extBot = tuple(c[c[:, :, 1].argmax()][0])
    ADD_PIXELS = 0
    new_img = img[extTop[1]-ADD_PIXELS:extBot[1]+ADD_PIXELS, extLeft[0]-ADD_PIXELS:extRight[0]+ADD_PIXELS].copy()
    
    return new_img

In [20]:
BATCH_SIZE = 64
NUM_EPOCHS = 20

transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((224, 224)),
    transforms.RandomRotation(10),
    transforms.ColorJitter(brightness=0.4, contrast=0.6),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [21]:
root_dir = './Training'

dataset = BrainTumorDataset(root_dir, transform=transform)
data_loader = DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=True)

In [ ]:
train_fraction = 0.8
validation_fraction = 0.1
test_fraction = 0.1
dataset_size = len(dataset)
[print(dataset_size)]

num_train = int(train_fraction * dataset_size)
num_validation = int(validation_fraction * dataset_size)
num_test = int(test_fraction * dataset_size)
print(num_train, num_validation, num_test)

train_dataset, validation_dataset, test_dataset = torch.utils.data.random_split(
    dataset, [num_train, num_validation, num_test]
)

In [22]:
class BrainTumorCNN(nn.Module):
    def __init__(self, num_classes=4):
        super(BrainTumorCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 16, 3)
        self.conv2 = nn.Conv2d(16, 32, 3)
        self.conv3 = nn.Conv2d(32, 64, 3)
        self.fc1 = nn.Linear(43264, 128)
        self.drop = nn.Dropout(0.5)

    def forward(self, x):
        x = nn.functional.relu(self.conv1(x))
        x = nn.functional.max_pool2d(x, 2)
        x = nn.functional.relu(self.conv2(x))
        x = nn.functional.max_pool2d(x, 2)
        x = nn.functional.relu(self.conv3(x))
        x = nn.functional.max_pool2d(x, 2)
        x = self.drop(x)

        # Dynamic calculation of the flattened size
        x_size = x.size()[1] * x.size()[2] * x.size()[3]
        
        x = x.view(-1, x_size)
        x = self.fc1(x)
        return x


In [23]:
model = BrainTumorCNN()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.00001)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=30, gamma=0.1)

In [24]:
for epoch in range(NUM_EPOCHS):
    all_labels = []
    all_predictions = []
    
    for batch_idx, (data, target) in enumerate(data_loader):
        # Zero the parameter gradients
        optimizer.zero_grad()
        # Forward pass
        output = model(data)
        loss = criterion(output, target)
        # Backward pass and optimization
        loss.backward()
        optimizer.step()
        scheduler.step()

        # Calculate metrics
        _, predicted = torch.max(output.data, 1)
        all_labels.extend(target.cpu().numpy())
        all_predictions.extend(predicted.cpu().numpy())

        print(f'Epoch [{epoch+1}/{NUM_EPOCHS}], Step [{batch_idx+1}/{len(data_loader)}], Loss: {loss.item():.4f}')

    # Metrics after each epoch
    accuracy = accuracy_score(all_labels, all_predictions)
    precision = precision_score(all_labels, all_predictions, average='weighted')
    recall = recall_score(all_labels, all_predictions, average='weighted')
    f1 = f1_score(all_labels, all_predictions, average='weighted')

    print(f'\nMetrics after Epoch {epoch + 1} - Accuracy: {accuracy:.4f}, Precision: {precision:.4f}, Recall: {recall:.4f}, F1: {f1:.4f}\n')


Epoch [1/20], Step [1/399], Loss: 4.8737
Epoch [1/20], Step [2/399], Loss: 4.8200
Epoch [1/20], Step [3/399], Loss: 4.7527
Epoch [1/20], Step [4/399], Loss: 4.7072
Epoch [1/20], Step [5/399], Loss: 4.6723
Epoch [1/20], Step [6/399], Loss: 4.6189
Epoch [1/20], Step [7/399], Loss: 4.5869
Epoch [1/20], Step [8/399], Loss: 4.5860
Epoch [1/20], Step [9/399], Loss: 4.4986
Epoch [1/20], Step [10/399], Loss: 4.4738
Epoch [1/20], Step [11/399], Loss: 4.4043
Epoch [1/20], Step [12/399], Loss: 4.2977
Epoch [1/20], Step [13/399], Loss: 4.2703
Epoch [1/20], Step [14/399], Loss: 4.2959
Epoch [1/20], Step [15/399], Loss: 4.1875
Epoch [1/20], Step [16/399], Loss: 4.1449
Epoch [1/20], Step [17/399], Loss: 4.1274
Epoch [1/20], Step [18/399], Loss: 4.0735
Epoch [1/20], Step [19/399], Loss: 4.0290
Epoch [1/20], Step [20/399], Loss: 3.9291
Epoch [1/20], Step [21/399], Loss: 3.8568
Epoch [1/20], Step [22/399], Loss: 3.8128
Epoch [1/20], Step [23/399], Loss: 3.8395
Epoch [1/20], Step [24/399], Loss: 3.8321
E

/Users/adisveletanlic/git/ml-project/.venv/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Epoch [2/20], Step [1/399], Loss: 3.1790
Epoch [2/20], Step [2/399], Loss: 3.2390
Epoch [2/20], Step [3/399], Loss: 3.1888
Epoch [2/20], Step [4/399], Loss: 3.1454
Epoch [2/20], Step [5/399], Loss: 3.2888
Epoch [2/20], Step [6/399], Loss: 3.1108
Epoch [2/20], Step [7/399], Loss: 3.1808
Epoch [2/20], Step [8/399], Loss: 3.1507
Epoch [2/20], Step [9/399], Loss: 3.1374
Epoch [2/20], Step [10/399], Loss: 3.3091
Epoch [2/20], Step [11/399], Loss: 3.2492
Epoch [2/20], Step [12/399], Loss: 3.1634
Epoch [2/20], Step [13/399], Loss: 3.1878
Epoch [2/20], Step [14/399], Loss: 3.1919
Epoch [2/20], Step [15/399], Loss: 3.1641
Epoch [2/20], Step [16/399], Loss: 3.2610
Epoch [2/20], Step [17/399], Loss: 3.2110
Epoch [2/20], Step [18/399], Loss: 3.2091
Epoch [2/20], Step [19/399], Loss: 3.2159
Epoch [2/20], Step [20/399], Loss: 3.2513
Epoch [2/20], Step [21/399], Loss: 3.1191
Epoch [2/20], Step [22/399], Loss: 3.2375
Epoch [2/20], Step [23/399], Loss: 3.1717
Epoch [2/20], Step [24/399], Loss: 3.1931
E

KeyboardInterrupt: 

In [ ]:
test_root_dir = './Testing'
test_dataset = BrainTumorDataset(test_root_dir, transform=transform)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

In [ ]:
# Test loop for evaluation
model.eval()
all_preds = []
all_labels = []

with torch.no_grad():
    for data, target in test_loader:
        output = model(data)
        _, predicted = torch.max(output.data, 1)
        all_preds.extend(predicted.cpu().numpy())
        all_labels.extend(target.cpu().numpy())

test_accuracy = accuracy_score(all_labels, all_preds)
print(f'Test Accuracy: {test_accuracy}')

In [ ]:
from keras.preprocessing.image import ImageDataGenerator
from keras.regularizers import l2
from keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D, BatchNormalization
from sklearn.utils.class_weight import compute_class_weight
import numpy as np


train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    zoom_range=0.2,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    vertical_flip=True,
    shear_range=0.2,
    fill_mode='nearest'
)

test_datagen = ImageDataGenerator(
    rescale=1./255,
)

train_generator = train_datagen.flow_from_directory(
    'Training/',
    target_size=(150, 150),
    color_mode='grayscale',
    batch_size=64,
    class_mode='categorical',
    seed=7
)

test_generator = test_datagen.flow_from_directory(
    'Testing/',
    target_size=(150, 150),
    color_mode='grayscale',
    batch_size=64,
    class_mode='categorical',
    seed=7
)

# Setting Up the Model

In [ ]:
# Model architecture
model = Sequential()

# 1st layer
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 1)))
model.add(MaxPooling2D((2, 2)))

# 2nd layer
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))

# 3rd layer
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))

# 4th layer
model.add(Conv2D(256, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))

# Flatten and fully connect Layers
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(4, activation='softmax'))

# Compile model
model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy', 'AUC']
)

model.summary()

In [ ]:
y_train = train_generator.classes
unique_classes = np.unique(y_train)

# Compute class weights
class_weights = compute_class_weight('balanced', classes=unique_classes, y=y_train)
class_weights_dict = dict(zip(unique_classes, class_weights))


In [ ]:
# Callbacks
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=1, verbose=1, min_lr=0.001)
model_checkpoint = ModelCheckpoint('best_model.h5', monitor='val_loss', save_best_only=True)

callbacks = [early_stopping, reduce_lr, model_checkpoint]

In [ ]:
history = model.fit(
    train_generator,
    epochs=20,
    validation_data=test_generator,
    callbacks=callbacks,
    class_weight=class_weights_dict
)

In [ ]:
import matplotlib.pyplot as plt

fig, axs = plt.subplots(3, 1, figsize=(10, 15))

points = {
    'accuracy': history.history['accuracy'],
    'val_accuracy': history.history['val_accuracy'],
    'loss': history.history['loss'],
    'val_loss': history.history['val_loss'],
    'AUC': history.history['auc'],
    'val_AUC': history.history['val_auc'],
}

# Plotting accuracy
axs[0].plot(points['accuracy'])
axs[0].plot(points['val_accuracy'])
axs[0].set_title('Model Accuracy')
axs[0].set_ylabel('Accuracy')
axs[0].set_xlabel('Epoch')
axs[0].legend(['Train', 'Test'], loc='upper left')

# Plotting loss
axs[1].plot(points['loss'])
axs[1].plot(points['val_loss'])
axs[1].set_title('Model Loss')
axs[1].set_ylabel('Loss')
axs[1].set_xlabel('Epoch')
axs[1].legend(['Train', 'Test'], loc='upper left')

# Plotting AUC
axs[2].plot(points['AUC'])
axs[2].plot(points['val_AUC'])
axs[2].set_title('Model AUC')
axs[2].set_ylabel('AUC')
axs[2].set_xlabel('Epoch')
axs[2].legend(['Train', 'Test'], loc='upper left')

plt.show()